In [1]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
alongtrack_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/')
alongtrack_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/')
alongtrack_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_missingdata/')
alongtrack_file_dir_orbiterror = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/')
alongtrack_file_dir_3errors = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/')
grids_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output/')
grids_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/')
grids_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/')
grids_file_dir_orbiterror = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/')
grids_file_dir_3errors = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/')
beckley_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/Beckley/')
main_dir = Path('/home/jovyan/ECCO_GMSL/')
output_dir = Path('/home/jovyan/Data/ECCO_GMSL/GMSL/')

# Compute Beckley along track GMSL
### Compute parameters

In [3]:
Vs = 7.2 #satellite along track velocity in km/s
Ve = 0.46 #velocity of earth's rotation at the equator in km/s
incidence = np.deg2rad(66.04) #satellite inclination
sig = 1 #uncertainty
alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(0)))
gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(0)) / (Vs*np.cos(alpha)) ))
beta0=np.pi/2 - gamma

#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [24]:
beckley_alongtrack_file_dir = list(beckley_file_dir.glob('Merged*Cycle*nc'))
beckley_alongtrack_file_dir=np.sort(beckley_alongtrack_file_dir)
beckley_alongtrack_file_dir

array([PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0001.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0002.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_0003.V5_1.nc'),
       ...,
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1057.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1058.V5_1.nc'),
       PosixPath('/home/jovyan/Data/ECCO_GMSL/Beckley/Merged_TOPEX_Jason_OSTM_Jason-3_Cycle_1059.V5_1.nc')],
      dtype=object)

In [25]:
beckley_gmsl_tmp=[]
for i in np.arange(0,len(beckley_alongtrack_file_dir)):
    alongtrack = xr.open_mfdataset(
            paths=beckley_alongtrack_file_dir[i],
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.ssha*10**(-3)
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    beckley_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
beckley_gmsl = xr.concat(beckley_gmsl_tmp, dim='time')

In [26]:
beckley_gmsl.name = 'gmsl_beckley_alongtrack'
beckley_gmsl.attrs['units'] = 'm'
beckley_gmsl.attrs['summary'] = '10-day GMSL time series computed on the alongtrack GSFC data'#, referenced to a 1996-2016 global mean SSH'
beckley_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
#we use the Beckley time series for the time
gmsl_beckley = xr.open_dataset(str(output_dir) + '/GMSL_Beckley_v51.nc')
beckley_gmsl['time']=gmsl_beckley.time[0:len(beckley_gmsl)]
beckley_gmsl

<xarray.DataArray 'gmsl_beckley_alongtrack' (time: 1058)>
dask.array<concatenate, shape=(1058,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1993-01-05 1993-01-15 ... 2021-09-25
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack GSFC data

In [28]:
fname = output_dir / ('GMSL_Beckley.nc')
beckley_gmsl.to_netcdf(fname)

# Compute ECCO along track GMSL (Beckley's style)

### No error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [11]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [12]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [13]:
# ref_mean=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean)

#### Compute GMSL 10-day time series

In [14]:
ECCO_alongtrack_files = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
# print(ECCO_alongtrack_files)

In [15]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
    
    # fig = plt.figure(figsize=(15,5))
    # cm = plt.get_cmap('gist_rainbow')
    # ax=fig.gca()
    # ax = plt.axes(projection=ccrs.Robinson( \
    #               central_longitude=-67, globe=None))
    # ax.gridlines()
    # ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.COASTLINE)
    # kk=12
    # p=ax.scatter(alongtrack.lon[::kk],\
    #                      alongtrack.lat[::kk], \
    #                      c=(SSH_at_xy_ano*w)[::kk], s=1,\
    #                      transform=ccrs.PlateCarree(),
    #                      vmin=-1,vmax=1, cmap='jet')
alongtrack_gmsl = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [16]:
SSH_at_xy_ano

<xarray.DataArray 'SSH_at_xy' (i: 59523)>
dask.array<where, shape=(59523,), dtype=float32, chunksize=(59523,), chunktype=numpy.ndarray>
Coordinates:
    time        datetime64[ns] 2017-12-31
    cycle_day   int64 0
    cycle       int64 924
    delta_days  int64 9230
    lon         (i) float32 dask.array<chunksize=(59523,), meta=np.ndarray>
    lat         (i) float32 dask.array<chunksize=(59523,), meta=np.ndarray>
Dimensions without coordinates: i
Attributes:
    source:   ECCO V4r4

In [17]:
alongtrack_gmsl=alongtrack_gmsl.drop('cycle_day')
alongtrack_gmsl=alongtrack_gmsl.drop('delta_days')
alongtrack_gmsl.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl.attrs['units'] = 'm'
alongtrack_gmsl.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
alongtrack_gmsl['time']=alongtrack_gmsl.time+(4*24*60*60*10**(9))
alongtrack_gmsl

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data

### Random noise error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [18]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [19]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [20]:
# ref_mean_randomnoise=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_randomnoise)

#### Compute GMSL 10-day time series

In [21]:
ECCO_alongtrack_files = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-23_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-24_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_1992-09-25_randomnoise.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-29_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-30_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise/ECCO_V4r4_alongtrack_SSH_2017-12-31_randomnoise.nc')]


In [22]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_randomnoise = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [23]:
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('cycle_day')
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('delta_days')
alongtrack_gmsl_randomnoise.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_randomnoise.attrs['units'] = 'm'
alongtrack_gmsl_randomnoise.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with random noise'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_randomnoise.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_randomnoise['time']=alongtrack_gmsl_randomnoise.time+(4*24*60*60*10**(9))
alongtrack_gmsl_randomnoise

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

### Missing data
#### Compute the reference global mean SSH over 20 years 1996-2016

In [24]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [25]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [26]:
# ref_mean_missingdata=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_missingdata)

#### Compute GMSL 10-day time series

In [27]:
ECCO_alongtrack_files = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
# print(ECCO_alongtrack_files)

In [28]:
alongtrack_gmsl_tmp_missingdata=[]
alongtrack_gmsl_tmp_missingdata2=[]
ind_vec=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano_missingdata=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w_missingdata=np.cos(np.deg2rad(alongtrack.lat))
    w_missingdata=w_missingdata.where(np.isfinite(SSH_at_xy_ano_missingdata))
    alongtrack_gmsl_tmp_missingdata.append((SSH_at_xy_ano_missingdata*w_missingdata).sum(skipna=True)/w_missingdata.sum(skipna=True))
alongtrack_gmsl_missingdata = xr.concat(alongtrack_gmsl_tmp_missingdata, dim='time')

In [29]:
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('cycle_day')
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('delta_days')
alongtrack_gmsl_missingdata.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_missingdata.attrs['units'] = 'm'
alongtrack_gmsl_missingdata.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with missing data'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_missingdata.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_missingdata['time']=alongtrack_gmsl_missingdata.time+(4*24*60*60*10**(9))
alongtrack_gmsl_missingdata

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

### Orbit error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [30]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_orbiterror.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [31]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [32]:
# ref_mean_orbiterror=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_orbiterror)

#### Compute GMSL 10-day time series

In [33]:
ECCO_alongtrack_files = list(alongtrack_file_dir_orbiterror.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_1992-09-23_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_1992-09-24_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_1992-09-25_orbiterror.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_2017-12-29_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_2017-12-30_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror/ECCO_V4r4_alongtrack_SSH_2017-12-31_orbiterror.nc')]


In [34]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_orbiterror = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [35]:
alongtrack_gmsl_orbiterror=alongtrack_gmsl_orbiterror.drop('cycle_day')
alongtrack_gmsl_orbiterror=alongtrack_gmsl_orbiterror.drop('delta_days')
alongtrack_gmsl_orbiterror.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_orbiterror.attrs['units'] = 'm'
alongtrack_gmsl_orbiterror.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with orbit error'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_orbiterror.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_orbiterror['time']=alongtrack_gmsl_orbiterror.time+(4*24*60*60*10**(9))
alongtrack_gmsl_orbiterror

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

### 3 errors: Random noise + missing data + orbit error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [36]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_orbiterror.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [37]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [38]:
# ref_mean_3errors=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_3errors)

#### Compute GMSL 10-day time series

In [39]:
ECCO_alongtrack_files = list(alongtrack_file_dir_3errors.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_1992-09-23_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_1992-09-24_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_1992-09-25_3errors.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_2017-12-29_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_2017-12-30_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors/ECCO_V4r4_alongtrack_SSH_2017-12-31_3errors.nc')]


In [40]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    # alpha = np.arcsin(np.abs(np.cos(incidence)/np.cos(np.deg2rad(alongtrack.lat))))
    # gamma = np.arctan(np.abs( (Vs*np.sin(alpha)-Ve*np.cos(np.deg2rad(alongtrack.lat))) / (Vs*np.cos(alpha)) ))
    # beta = np.pi/2 - gamma
    # w=np.sin(beta)/np.sin(beta0) #From Nerem 1995
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_3errors = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [41]:
alongtrack_gmsl_3errors=alongtrack_gmsl_3errors.drop('cycle_day')
alongtrack_gmsl_3errors=alongtrack_gmsl_3errors.drop('delta_days')
alongtrack_gmsl_3errors.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_3errors.attrs['units'] = 'm'
alongtrack_gmsl_3errors.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3 errors'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_3errors.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_3errors['time']=alongtrack_gmsl_3errors.time+(4*24*60*60*10**(9))
alongtrack_gmsl_3errors

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

### Save to Disk

In [42]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_noerror_simpleweight_nomean.nc')
alongtrack_gmsl.to_netcdf(fname)

In [43]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_randomnoise_simpleweight_nomean.nc')
alongtrack_gmsl_randomnoise.to_netcdf(fname)

In [44]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_missingdata_simpleweight_nomean.nc')
alongtrack_gmsl_missingdata.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4705: RuntimeWarning: invalid value encountered in float_scalars
  result = function(*args, **kwargs)


In [45]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_orbiterror_simpleweight_nomean.nc')
alongtrack_gmsl_orbiterror.to_netcdf(fname)

In [46]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_3errors_simpleweight_nomean.nc')
alongtrack_gmsl_3errors.to_netcdf(fname)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/core.py:4705: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)


# Compute GMSL from ECCO synthetic grids

### No error

In [47]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output/SSHA_gridded_2017-12-25.nc')]


In [48]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [49]:
grids=grids.where(np.array(grids.counts)>=100)

In [50]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [51]:
# make new DataArray objects
gmsl_grids = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids = gmsl_grids.assign_coords({'time':grids.time})
gmsl_grids.name = 'gmsl_grids'
gmsl_grids

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.31269167e-02, -3.15490700e-02, -2.98427232e-02, -3.02407369e-02,
       -3.10245752e-02, -3.27937305e-02, -3.13237682e-02, -3.23810466e-02,
       -3.31036970e-02, -3.83624732e-02, -3.98116633e-02, -3.93596627e-02,
       -4.03957106e-02, -4.20353301e-02, -4.22189347e-02, -4.00765687e-02,
       -3.96159030e-02, -3.79023999e-02, -4.01852168e-02, -3.84850949e-02,
       -3.78896035e-02, -3.74921635e-02, -3.86221148e-02, -3.83689329e-02,
       -3.70318219e-02, -3.64701971e-02, -3.78421992e-02, -3.73318158e-02,
       -3.86409983e-02, -3.78626473e-02, -3.66033763e-02, -3.69629078e-02,
       -3.44672948e-02, -3.35801654e-02, -3.22836228e-02, -3.17521468e-02,
       -3.01833730e-02, -2.81687696e-02, -2.75812577e-02, -2.72999387e-02,
       -2.89914161e-02, -3.00419126e-02, -3.22561301e-02, -3.14620584e-02,
       -2.82080099e-02, -2.67691519e-02, -3.10667269e-02, -3.18353809e-02,
       -3.03786527e-02, -3.35939080e-02, -3.77003960e-02, -3.86640355e-02,
       -3.47501971e-02, -3.35698612e-02, -3.22869904e-02, -3.31326611e-02,
       -3.22120972e-02, -3.29493284e-02, -3.36165726e-02, -3.62860411e-02,
       -3.53421010e-02, -3.48336846e-02, -3.79533879e-02, -3.88327055e-02,
       -3.91413756e-02, -3.83893140e-02, -3.67083885e-02, -3.66187543e-02,
       -3.54870930e-02, -3.50018181e-02, -3.18838581e-02, -3.20794880e-02,
       -2.95780860e-02, -2.86077131e-02, -2.64548007e-02, -2.71791462e-02,
       -2.50093695e-02, -2.64327545e-02, -2.70502456e-02, -2.75706705e-02,
...
        4.39499393e-02,  4.00665253e-02,  3.86737101e-02,  4.25332151e-02,
        4.15377840e-02,  3.82172279e-02,  3.83247100e-02,  3.69742103e-02,
        3.84840816e-02,  3.59458476e-02,  3.82077359e-02,  3.58815864e-02,
        3.70455682e-02,  3.72231118e-02,  3.89818698e-02,  3.85158025e-02,
        3.64143737e-02,  3.78729068e-02,  3.70380059e-02,  3.66132893e-02,
        3.68282348e-02,  3.64439376e-02,  3.50860804e-02,  3.88407521e-02,
        3.92371193e-02,  3.99822295e-02,  4.12110165e-02,  4.23920564e-02,
        4.24831733e-02,  4.25542444e-02,  4.39601727e-02,  4.65233661e-02,
        4.58209701e-02,  4.83235791e-02,  4.75424416e-02,  4.91971262e-02,
        4.74252366e-02,  4.71487679e-02,  4.53022197e-02,  4.54813763e-02,
        4.30943444e-02,  3.89054306e-02,  3.75446118e-02,  3.78499068e-02,
        3.76777127e-02,  3.80793624e-02,  3.76724042e-02,  3.78213935e-02,
        3.89150009e-02,  3.99266444e-02,  4.05345634e-02,  4.11128849e-02,
        3.92694660e-02,  3.96253653e-02,  4.20884416e-02,  4.13078703e-02,
        4.16005291e-02,  3.98898646e-02,  4.11925875e-02,  4.26474214e-02,
        4.39943746e-02,  4.60651405e-02,  4.66651656e-02,  4.72973622e-02,
        4.90671322e-02,  4.69386987e-02,  4.84141372e-02,  4.91192192e-02,
        5.22320904e-02,  5.35467863e-02,  5.39420173e-02,  5.60682751e-02,
        5.46174049e-02,  5.55606857e-02,  5.40054105e-02,  5.54823950e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Random noise

In [52]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_randomnoise.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise/SSHA_gridded_2017-12-25.nc')]


In [53]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [54]:
grids=grids.where(np.array(grids.counts)>=100)

In [55]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [56]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [57]:
# ssh_weighted_mean_randomnoise = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_randomnoise.SSHA

In [58]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_randomnoise=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [59]:
# make new DataArray objects
gmsl_grids_randomnoise = xr.DataArray(ssh_weighted_mean_randomnoise, dims=['time'])
gmsl_grids_randomnoise = gmsl_grids_randomnoise.assign_coords({'time':grids.time})
gmsl_grids_randomnoise.name = 'gmsl_grids'
gmsl_grids_randomnoise

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.31307389e-02, -3.15057933e-02, -2.98172422e-02, -3.02942675e-02,
       -3.09962742e-02, -3.28599103e-02, -3.13217826e-02, -3.24213579e-02,
       -3.30449380e-02, -3.83874737e-02, -3.97898890e-02, -3.92875411e-02,
       -4.04346138e-02, -4.20676284e-02, -4.22190689e-02, -4.00915928e-02,
       -3.95240709e-02, -3.79373915e-02, -4.02326882e-02, -3.84830274e-02,
       -3.79466340e-02, -3.74614447e-02, -3.86067443e-02, -3.83415334e-02,
       -3.70063111e-02, -3.64461839e-02, -3.78557667e-02, -3.73040736e-02,
       -3.85795049e-02, -3.79438587e-02, -3.65840904e-02, -3.70159671e-02,
       -3.44327316e-02, -3.35680209e-02, -3.23160701e-02, -3.16621512e-02,
       -3.02907266e-02, -2.81298403e-02, -2.75555383e-02, -2.73336619e-02,
       -2.90293396e-02, -3.00987512e-02, -3.23145874e-02, -3.14850435e-02,
       -2.82234568e-02, -2.67909430e-02, -3.11236437e-02, -3.18637714e-02,
       -3.04112174e-02, -3.35414894e-02, -3.76984179e-02, -3.86905298e-02,
       -3.47107388e-02, -3.35463099e-02, -3.23002711e-02, -3.31086852e-02,
       -3.22280452e-02, -3.29552256e-02, -3.35866585e-02, -3.63279805e-02,
       -3.52951139e-02, -3.48416828e-02, -3.79239321e-02, -3.88001315e-02,
       -3.91480736e-02, -3.83434184e-02, -3.66633199e-02, -3.66593488e-02,
       -3.55160497e-02, -3.49957682e-02, -3.18268985e-02, -3.20711099e-02,
       -2.95471158e-02, -2.86911856e-02, -2.64440570e-02, -2.71383021e-02,
       -2.49777772e-02, -2.64221877e-02, -2.70567629e-02, -2.75493748e-02,
...
        4.39405628e-02,  4.00604978e-02,  3.87389325e-02,  4.25249226e-02,
        4.15033549e-02,  3.82632501e-02,  3.83557566e-02,  3.69893983e-02,
        3.84428278e-02,  3.59715447e-02,  3.82392257e-02,  3.59523594e-02,
        3.71370390e-02,  3.72597128e-02,  3.90274599e-02,  3.84870581e-02,
        3.63432914e-02,  3.79256383e-02,  3.70186940e-02,  3.66587155e-02,
        3.68283093e-02,  3.64706814e-02,  3.50907780e-02,  3.88605781e-02,
        3.91399115e-02,  4.00102735e-02,  4.12400961e-02,  4.23319936e-02,
        4.24517095e-02,  4.24906500e-02,  4.38748114e-02,  4.65457812e-02,
        4.58504036e-02,  4.83129546e-02,  4.75587063e-02,  4.92359176e-02,
        4.74531390e-02,  4.72370051e-02,  4.53138202e-02,  4.54823300e-02,
        4.30806652e-02,  3.89296003e-02,  3.75324786e-02,  3.78173739e-02,
        3.77049297e-02,  3.80897820e-02,  3.75946872e-02,  3.78674492e-02,
        3.89678217e-02,  3.99146825e-02,  4.05153744e-02,  4.10319716e-02,
        3.92988473e-02,  3.96106318e-02,  4.21217196e-02,  4.13434729e-02,
        4.15543728e-02,  3.99514176e-02,  4.11466621e-02,  4.26615328e-02,
        4.39280793e-02,  4.61032130e-02,  4.66491617e-02,  4.72645275e-02,
        4.90047820e-02,  4.69401889e-02,  4.84670065e-02,  4.90854234e-02,
        5.21991067e-02,  5.35118915e-02,  5.38475364e-02,  5.60598373e-02,
        5.45547828e-02,  5.55159785e-02,  5.40253967e-02,  5.54666370e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Missing data

In [60]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_missingdata.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata/SSHA_gridded_2017-12-25.nc')]


In [61]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [62]:
grids=grids.where(np.array(grids.counts)>=100)

In [63]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [64]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [65]:
# ssh_weighted_mean_missingdata = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_missingdata.SSHA

In [66]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_missingdata=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [67]:
# make new DataArray objects
gmsl_grids_missingdata = xr.DataArray(ssh_weighted_mean_missingdata, dims=['time'])
gmsl_grids_missingdata = gmsl_grids_missingdata.assign_coords({'time':grids.time})
gmsl_grids_missingdata.name = 'gmsl_grids'
gmsl_grids_missingdata

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.33564803e-02, -3.17081288e-02, -3.03789973e-02, -3.04863323e-02,
       -3.12455725e-02, -3.34679745e-02, -3.17645781e-02, -3.29496861e-02,
       -3.27363759e-02, -3.83356623e-02, -3.95934209e-02, -3.90708409e-02,
       -4.00482416e-02, -4.14513387e-02, -4.20339629e-02, -3.98834273e-02,
       -3.93723138e-02, -3.73586789e-02, -4.00248840e-02, -3.81970927e-02,
       -3.76659855e-02, -3.72205675e-02, -3.82492132e-02, -3.84214297e-02,
       -3.70715223e-02, -3.64140496e-02, -3.79666686e-02, -3.71045731e-02,
       -3.89354900e-02, -3.75830159e-02, -3.55910510e-02, -3.69768105e-02,
       -3.44920941e-02, -3.31916660e-02, -3.20920087e-02, -3.19158398e-02,
       -3.00563332e-02, -2.81936899e-02, -2.73213200e-02, -2.71080397e-02,
       -2.92159580e-02, -3.02211400e-02, -3.29696164e-02, -3.14168744e-02,
       -2.84057576e-02, -2.64727119e-02, -3.08137815e-02, -3.16509753e-02,
       -2.97527146e-02, -3.31507660e-02, -3.77203822e-02, -3.89055721e-02,
       -3.44105251e-02, -3.29243354e-02, -3.19771282e-02, -3.28548141e-02,
       -3.16963792e-02, -3.28079388e-02, -3.31811868e-02, -3.63533907e-02,
       -3.55096720e-02, -3.48919258e-02, -3.79535258e-02, -3.92269380e-02,
       -4.01537344e-02, -3.84392440e-02, -3.69637720e-02, -3.64349633e-02,
       -3.54692377e-02, -3.52957733e-02, -3.20884548e-02, -3.16311195e-02,
       -2.92605516e-02, -2.97650751e-02, -2.64315289e-02, -2.72088274e-02,
       -2.55076215e-02, -2.64581256e-02, -2.84607075e-02, -2.78352797e-02,
...
        4.61399034e-02,  4.71310988e-02,  4.46018502e-02,  4.34804894e-02,
        3.95865999e-02,  3.85096297e-02,  4.24883887e-02,  4.09164689e-02,
        3.78324315e-02,  3.82652171e-02,  3.66046093e-02,  3.88494544e-02,
        3.59697081e-02,  3.84957902e-02,  3.59645933e-02,  3.72774787e-02,
        3.73284109e-02,  3.91522311e-02,  3.84793840e-02,  3.66781205e-02,
        3.78764458e-02,  3.69907729e-02,  3.67235094e-02,  3.69237252e-02,
        3.66048366e-02,  3.52888778e-02,  3.88893187e-02,  3.94927301e-02,
        4.00558002e-02,  4.18093652e-02,  4.32090648e-02,  4.24931049e-02,
        4.27203141e-02,  4.42920849e-02,  4.69149612e-02,  4.62655462e-02,
        4.81569953e-02,  4.75375131e-02,  4.90223877e-02,  4.70583811e-02,
        4.67509367e-02,  4.50642295e-02,  4.51532789e-02,  4.24792469e-02,
        3.88399288e-02,  3.73848155e-02,  3.79597396e-02,  3.76473740e-02,
        3.81528921e-02,  3.79862301e-02,  3.80703434e-02,  3.89330871e-02,
        4.01686020e-02,  4.08212841e-02,  4.11185957e-02,  3.93476114e-02,
        3.95925939e-02,  4.21719514e-02,  4.12734523e-02,  4.12413143e-02,
        3.97732183e-02,  4.10904326e-02,  4.27514054e-02,  4.40126210e-02,
        4.60716449e-02,  4.66792881e-02,  4.73206230e-02,  4.94720973e-02,
        4.70931381e-02,  4.84317802e-02,  4.91705947e-02,  5.23814559e-02,
        5.37045039e-02,  5.39096110e-02,  5.60513958e-02,  5.46252467e-02,
        5.54327518e-02,  5.36909997e-02,  5.42162210e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Orbit Error

In [4]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror/SSHA_gridded_2017-12-25.nc')]


In [5]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [6]:
grids=grids.where(np.array(grids.counts)>=100)

In [7]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [8]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [9]:
# ssh_weighted_mean_orbiterror = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_orbiterror.SSHA

In [10]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [11]:
# make new DataArray objects
gmsl_grids_orbiterror = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror = gmsl_grids_orbiterror.assign_coords({'time':grids.time})
gmsl_grids_orbiterror.name = 'gmsl_grids'
gmsl_grids_orbiterror

<xarray.DataArray 'gmsl_grids' (time: 924)>
array([-3.32809500e-02, -3.13834623e-02, -2.97238603e-02, -3.00344694e-02,
       -3.10232770e-02, -3.26834023e-02, -3.11975442e-02, -3.25007029e-02,
       -3.34046930e-02, -3.85388024e-02, -3.99284102e-02, -3.94947119e-02,
       -4.06672694e-02, -4.20704521e-02, -4.21403274e-02, -3.99228595e-02,
       -3.93643305e-02, -3.79059725e-02, -4.01622988e-02, -3.84871364e-02,
       -3.79280373e-02, -3.74052376e-02, -3.87385674e-02, -3.83326970e-02,
       -3.70057523e-02, -3.63531820e-02, -3.79653275e-02, -3.75786200e-02,
       -3.85193713e-02, -3.80065553e-02, -3.65524665e-02, -3.69488001e-02,
       -3.44529338e-02, -3.35385129e-02, -3.22923474e-02, -3.18056978e-02,
       -3.04448754e-02, -2.83307489e-02, -2.78411917e-02, -2.74305493e-02,
       -2.87285987e-02, -3.02222949e-02, -3.22782546e-02, -3.12584452e-02,
       -2.81729791e-02, -2.69356761e-02, -3.13862264e-02, -3.18517983e-02,
       -3.01807933e-02, -3.35736163e-02, -3.79586741e-02, -3.87296453e-02,
       -3.46763767e-02, -3.35273333e-02, -3.23719010e-02, -3.30510400e-02,
       -3.22911888e-02, -3.28580663e-02, -3.34799401e-02, -3.63806821e-02,
       -3.54497768e-02, -3.50445807e-02, -3.79719324e-02, -3.86141278e-02,
       -3.90194096e-02, -3.82895768e-02, -3.66611555e-02, -3.64770256e-02,
       -3.57719809e-02, -3.52246277e-02, -3.19697484e-02, -3.19781899e-02,
       -2.97847912e-02, -2.85820793e-02, -2.63068713e-02, -2.73162797e-02,
       -2.49742512e-02, -2.63159368e-02, -2.72401851e-02, -2.72676814e-02,
...
        4.38850597e-02,  4.00677882e-02,  3.84167917e-02,  4.25271727e-02,
        4.14020568e-02,  3.81269529e-02,  3.81093509e-02,  3.70227434e-02,
        3.84309255e-02,  3.60968076e-02,  3.77120189e-02,  3.56049277e-02,
        3.69120017e-02,  3.73436920e-02,  3.89694981e-02,  3.84707972e-02,
        3.63985226e-02,  3.79709154e-02,  3.68399918e-02,  3.65795232e-02,
        3.67901437e-02,  3.60187553e-02,  3.52398492e-02,  3.89409214e-02,
        3.92726883e-02,  3.99731882e-02,  4.14390936e-02,  4.22739126e-02,
        4.24555354e-02,  4.23169509e-02,  4.40514274e-02,  4.65476774e-02,
        4.60060425e-02,  4.82770242e-02,  4.75352556e-02,  4.93900292e-02,
        4.73512895e-02,  4.71975021e-02,  4.50596325e-02,  4.55766656e-02,
        4.32008803e-02,  3.87784876e-02,  3.76657620e-02,  3.79171558e-02,
        3.75566259e-02,  3.83559056e-02,  3.75466049e-02,  3.78401428e-02,
        3.90621126e-02,  3.96390483e-02,  4.06926945e-02,  4.10255529e-02,
        3.91872451e-02,  3.96970995e-02,  4.19283472e-02,  4.10871617e-02,
        4.16701809e-02,  3.99697423e-02,  4.09730822e-02,  4.26621325e-02,
        4.41824347e-02,  4.60291989e-02,  4.65823747e-02,  4.71271314e-02,
        4.90037091e-02,  4.69598360e-02,  4.82801832e-02,  4.89847586e-02,
        5.19928560e-02,  5.37573956e-02,  5.39262705e-02,  5.60403243e-02,
        5.46489432e-02,  5.54446056e-02,  5.37803993e-02,  5.64283319e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### 3 errors: random noise + missing data + orbit error

In [12]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors/SSHA_gridded_2017-12-25.nc')]


In [13]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [15]:
grids=grids.where(np.array(grids.counts)>=100)

In [16]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [17]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [18]:
# ssh_weighted_mean_3errors = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_3errors.SSHA

In [19]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [20]:
# make new DataArray objects
gmsl_grids_3errors = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors = gmsl_grids_3errors.assign_coords({'time':grids.time})
gmsl_grids_3errors.name = 'gmsl_grids'
gmsl_grids_3errors

<xarray.DataArray 'gmsl_grids' (time: 923)>
array([-3.32398601e-02, -3.14501710e-02, -3.04541644e-02, -3.06259505e-02,
       -3.12125497e-02, -3.35333459e-02, -3.23746540e-02, -3.29280421e-02,
       -3.26760449e-02, -3.82078029e-02, -3.97380143e-02, -3.90054435e-02,
       -4.01074812e-02, -4.12344374e-02, -4.21246067e-02, -3.97941992e-02,
       -3.94912399e-02, -3.75352316e-02, -4.01957929e-02, -3.82857919e-02,
       -3.78761142e-02, -3.70437726e-02, -3.80724408e-02, -3.86117063e-02,
       -3.71989086e-02, -3.64866294e-02, -3.79813947e-02, -3.75468619e-02,
       -3.90482061e-02, -3.74335460e-02, -3.51898558e-02, -3.68566550e-02,
       -3.47619727e-02, -3.31513323e-02, -3.19886915e-02, -3.19396406e-02,
       -2.98821349e-02, -2.83513889e-02, -2.74745356e-02, -2.72437837e-02,
       -2.95336880e-02, -3.01252417e-02, -3.31665985e-02, -3.12734582e-02,
       -2.86428519e-02, -2.64476165e-02, -3.07443421e-02, -3.16421874e-02,
       -2.97018420e-02, -3.30579281e-02, -3.79933529e-02, -3.89845110e-02,
       -3.40713821e-02, -3.30636874e-02, -3.17126997e-02, -3.30321118e-02,
       -3.15873697e-02, -3.28500867e-02, -3.32394317e-02, -3.62991691e-02,
       -3.50047760e-02, -3.49046439e-02, -3.78782898e-02, -3.91424187e-02,
       -4.03058417e-02, -3.85867581e-02, -3.71403135e-02, -3.64692435e-02,
       -3.53910066e-02, -3.53795737e-02, -3.25123742e-02, -3.17208059e-02,
       -2.96255592e-02, -2.96174567e-02, -2.62635015e-02, -2.76320931e-02,
       -2.55987737e-02, -2.66454015e-02, -2.84256488e-02, -2.75690425e-02,
...
        4.58966158e-02,  4.68750782e-02,  4.48007062e-02,  4.36034016e-02,
        3.94897684e-02,  3.85719500e-02,  4.24527340e-02,  4.08487990e-02,
        3.79076563e-02,  3.86168696e-02,  3.68133858e-02,  3.85789424e-02,
        3.59118655e-02,  3.86322960e-02,  3.63165922e-02,  3.71810608e-02,
        3.70952934e-02,  3.89442071e-02,  3.84553038e-02,  3.68429758e-02,
        3.80172655e-02,  3.69265527e-02,  3.69340815e-02,  3.68507765e-02,
        3.64605300e-02,  3.50512639e-02,  3.88957486e-02,  3.97109762e-02,
        3.99407931e-02,  4.19044159e-02,  4.35988083e-02,  4.25668359e-02,
        4.28151190e-02,  4.44934666e-02,  4.68507521e-02,  4.64821979e-02,
        4.83607501e-02,  4.76432405e-02,  4.92231883e-02,  4.70795147e-02,
        4.69154306e-02,  4.50380221e-02,  4.51110117e-02,  4.25543711e-02,
        3.88820209e-02,  3.75077277e-02,  3.79297361e-02,  3.79337445e-02,
        3.81105095e-02,  3.78575549e-02,  3.78556177e-02,  3.88242900e-02,
        4.01847437e-02,  4.07689922e-02,  4.11798880e-02,  3.91818322e-02,
        3.99176702e-02,  4.20240015e-02,  4.15224917e-02,  4.12096903e-02,
        3.98604758e-02,  4.10395749e-02,  4.26795781e-02,  4.41909023e-02,
        4.62452844e-02,  4.67569791e-02,  4.71611880e-02,  4.93339449e-02,
        4.71059009e-02,  4.84764464e-02,  4.89547551e-02,  5.22618666e-02,
        5.34392297e-02,  5.37237413e-02,  5.60611673e-02,  5.43665588e-02,
        5.55281341e-02,  5.35459965e-02,  5.46690933e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Save to Disk

In [68]:
fname = output_dir / ('GMSL_grids_19922017_noerror_100points.nc')
gmsl_grids.to_netcdf(fname)

In [69]:
fname = output_dir / ('GMSL_grids_19922017_randomnoise_100points.nc')
gmsl_grids_randomnoise.to_netcdf(fname)

In [70]:
fname = output_dir / ('GMSL_grids_19922017_missingdata_100points.nc')
gmsl_grids_missingdata.to_netcdf(fname)

In [21]:
fname = output_dir / ('GMSL_grids_19922017_orbiterror_100points.nc')
gmsl_grids_orbiterror.to_netcdf(fname)

In [22]:
fname = output_dir / ('GMSL_grids_19922017_3errors_100points.nc')
gmsl_grids_3errors.to_netcdf(fname)

# Compute GMSL from daily ECCO output

### Compute the ocean area, needed to compute GMSL from ECCO

In [29]:
ECCO_grid = xr.open_dataset(str(main_dir) + '/GRID_GEOMETRY_ECCO_V4r4_latlon_0p50deg.nc')
ECCO_grid

<xarray.Dataset>
Dimensions:         (Z: 50, latitude: 360, longitude: 720, nv: 2)
Coordinates:
  * Z               (Z) float32 -5.0 -15.0 -25.0 ... -5.461e+03 -5.906e+03
  * latitude        (latitude) float32 -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
    latitude_bnds   (latitude, nv) float32 ...
    longitude_bnds  (longitude, nv) float32 ...
    Z_bnds          (Z, nv) float32 ...
Dimensions without coordinates: nv
Data variables:
    hFacC           (Z, latitude, longitude) float64 ...
    Depth           (latitude, longitude) float64 ...
    area            (latitude, longitude) float64 ...
    drF             (Z) float32 ...
    maskC           (Z, latitude, longitude) bool ...
Attributes: (12/57)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on a regular lat-lon gri...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    references:                      ECCO Consortium, Fukumori, I., Wang, O.,...
    source:                          The ECCO V4r4 state estimate was produce...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides geometric paramete...
    title:                           ECCO Geometry Parameters for the 0.5 deg...
    uuid:                            b4795c62-86e5-11eb-9c5f-f8f21e2ee3e0

In [30]:
# mask land points
ECCO_ocean_area = ECCO_grid.area*ECCO_grid.maskC[0,:]
ECCO_ocean_area=ECCO_ocean_area.drop('Z')

ECCO_total_ocean_area = ECCO_ocean_area.sum(skipna=True).values

### Get the list of ECCO daily data in the cloud

In [31]:
%%capture
import requests

def store_aws_keys(endpoint: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):    
    with requests.get(endpoint, "w") as r:
        accessKeyId, secretAccessKey, sessionToken, expiration = list(r.json().values())

    creds ={}
    creds['AccessKeyId'] = accessKeyId
    creds['SecretAccessKey'] = secretAccessKey
    creds['SessionToken'] = sessionToken
    creds['expiration'] = expiration
    
    return creds

creds = store_aws_keys()
print(creds)

In [32]:
ShortName = "ECCO_L4_SSH_05DEG_DAILY_V4R4B"
# Ask PODAAC for the collection id
response = requests.get(
    url='https://cmr.earthdata.nasa.gov/search/collections.umm_json', 
    params={'provider': "POCLOUD",
            'ShortName': ShortName,
            'page_size': 1}
)
ummc = response.json()['items']
# ummc
# used to work, doesn't work anymore...
# ummc = response.json()['items'][0]
# ccid = ummc['meta']['concept-id']
# print(f'collection id: {ccid}')

In [33]:
s3 = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

In [34]:
# make a S3 'filesystem' object
fs = s3fs.S3FileSystem(anon=False,
                      key=creds['AccessKeyId'],
                      secret=creds['SecretAccessKey'],
                      token=creds['SessionToken'])

### Read ECCO data from the cloud

In [35]:
ECCO_SSH_files = fs.glob(join("podaac-ops-cumulus-protected/", ShortName, '*.nc'))
ECCO_SSH_files

['podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-01_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-02_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-03_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-04_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-05_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-06_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day_mean_1992-01-07_ECCO_V4r4b_latlon_0p50deg.nc',
 'podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4B/SEA_SURFACE_HEIGHT_day

In [38]:
#we only start on 09-23-1992, doy=266
ECCO_SSH_files=ECCO_SSH_files[266:]

In [39]:
#Load all of the files from AWS S3 using 'direct connection' and combine into a single xarray DataSet object
paths=[fs.open(f) for f in ECCO_SSH_files] 
ECCO_DS_daily = xr.open_mfdataset(
    paths=paths,
    combine='by_coords',
    # concat_dim='time',
    decode_cf=True,
    coords='minimal',
    chunks={'time': 1}  
)
ECCO_DS_daily.close()
ECCO_DS_daily

<xarray.Dataset>
Dimensions:         (time: 9231, latitude: 360, longitude: 720, nv: 2)
Coordinates:
  * time            (time) datetime64[ns] 1992-09-23T12:00:00 ... 2017-12-31T...
  * latitude        (latitude) float32 -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
    time_bnds       (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    latitude_bnds   (latitude, nv) float32 dask.array<chunksize=(360, 2), meta=np.ndarray>
    longitude_bnds  (longitude, nv) float32 dask.array<chunksize=(720, 2), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables:
    SSH             (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHIBC          (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHNOIBC        (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes: (12/57)
    acknowledgement:              This research was carried out by the Jet Pr...
    author:                       Ou Wang and Ian Fenty
    cdm_data_type:                Grid
    comment:                      Fields provided on a regular lat-lon grid. ...
    Conventions:                  CF-1.8, ACDD-1.3
    coordinates_comment:          Note: the global 'coordinates' attribute de...
    ...                           ...
    time_coverage_duration:       P1D
    time_coverage_end:            1992-09-24T00:00:00
    time_coverage_resolution:     P1D
    time_coverage_start:          1992-09-23T00:00:00
    title:                        ECCO Sea Surface Height - Daily Mean 0.5 De...
    uuid:                         44a9c25e-9a8a-414c-85e0-ebe688de113d

In [40]:
ECCO_DS_10days=ECCO_DS_daily.resample(time="10D",loffset=datetime.timedelta(days=4)).mean()
ECCO_DS_10days

<xarray.Dataset>
Dimensions:         (time: 924, latitude: 360, longitude: 720, nv: 2)
Coordinates:
  * time            (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
  * latitude        (latitude) float32 -89.75 -89.25 -88.75 ... 89.25 89.75
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
    latitude_bnds   (latitude, nv) float32 dask.array<chunksize=(360, 2), meta=np.ndarray>
    longitude_bnds  (longitude, nv) float32 dask.array<chunksize=(720, 2), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables:
    SSH             (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHIBC          (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    SSHNOIBC        (time, latitude, longitude) float32 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

### Extract SSH

In [41]:
ECCO_SSH = ECCO_DS_10days.SSH
ECCO_SSH

<xarray.DataArray 'SSH' (time: 924, latitude: 360, longitude: 720)>
dask.array<concatenate, shape=(924, 360, 720), dtype=float32, chunksize=(1, 360, 720), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
  * latitude   (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float32 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8

### Calculate the True daily GMSL from ECCO

In [42]:
# first call sets up the calculation in dask
# ... \sum_i [SSH_i x grid cell area_i] / total grid cell area
ECCO_global_mean_sea_level = (ECCO_SSH * ECCO_ocean_area).sum(dim=['latitude','longitude'],skipna=True)/ECCO_total_ocean_area

# second call actually computes it
ECCO_global_mean_sea_level = ECCO_global_mean_sea_level.compute()

In [43]:
ECCO_global_mean_sea_level

<xarray.DataArray (time: 924)>
array([0.11195291, 0.11285948, 0.11341337, 0.11300809, 0.11313697,
       0.11264513, 0.11223841, 0.11163162, 0.11048567, 0.10895309,
       0.10735631, 0.10700142, 0.10556312, 0.1032998 , 0.10361039,
       0.10383226, 0.10407618, 0.10511998, 0.10563994, 0.10649896,
       0.10635719, 0.10615614, 0.10575123, 0.10613427, 0.10730206,
       0.1077906 , 0.10674654, 0.10643219, 0.10623923, 0.10702848,
       0.10796349, 0.10916335, 0.11039218, 0.11110167, 0.11227004,
       0.1144013 , 0.11563627, 0.11646839, 0.11693976, 0.11731911,
       0.11742691, 0.11671939, 0.11617301, 0.1160349 , 0.11568373,
       0.11501275, 0.11381891, 0.11327246, 0.11210064, 0.11027458,
       0.10938764, 0.10881524, 0.10922673, 0.10972002, 0.11053886,
       0.11017111, 0.11110017, 0.11079002, 0.1100847 , 0.10915513,
       0.10879783, 0.10881178, 0.10736755, 0.10675891, 0.10691611,
       0.1075518 , 0.10851055, 0.10987104, 0.11085093, 0.11148839,
       0.11317652, 0.11443962, 0.11493115, 0.11651236, 0.11792628,
       0.11883701, 0.11969608, 0.11949465, 0.11882656, 0.11872376,
       0.1175553 , 0.11651506, 0.11581397, 0.11516827, 0.1143025 ,
       0.11413427, 0.11376543, 0.11313991, 0.11382699, 0.11375053,
       0.11323009, 0.11359898, 0.11345613, 0.1125047 , 0.11197785,
       0.11151998, 0.11042972, 0.11094996, 0.11166883, 0.11098661,
...
       0.17834888, 0.17841647, 0.17724194, 0.17697176, 0.17766824,
       0.17804981, 0.178273  , 0.17944553, 0.17976841, 0.18059395,
       0.18233745, 0.18400141, 0.18673113, 0.18875708, 0.19051162,
       0.19289356, 0.19391164, 0.1950074 , 0.19544065, 0.1938877 ,
       0.19368028, 0.19241159, 0.19154215, 0.19082609, 0.18903911,
       0.18861511, 0.18867342, 0.18702611, 0.18473693, 0.18405277,
       0.18323214, 0.1830952 , 0.18226264, 0.18193957, 0.18160507,
       0.18128057, 0.18255383, 0.18342646, 0.1835636 , 0.18331193,
       0.18320504, 0.18349212, 0.18357898, 0.18360235, 0.18352895,
       0.18345358, 0.18493529, 0.18617877, 0.1876177 , 0.1886434 ,
       0.18903164, 0.19025885, 0.19221552, 0.19379673, 0.19439059,
       0.19455476, 0.19509852, 0.19547613, 0.19517602, 0.1956606 ,
       0.19444977, 0.19322036, 0.19210036, 0.18969334, 0.18755868,
       0.18648735, 0.18480059, 0.18386856, 0.1832963 , 0.18341435,
       0.18356444, 0.18423185, 0.18507829, 0.18610567, 0.18568242,
       0.18552377, 0.18576799, 0.18677459, 0.18680344, 0.18690166,
       0.18697234, 0.18819596, 0.18894286, 0.19048129, 0.19124022,
       0.1925163 , 0.19393949, 0.1950036 , 0.19517333, 0.19541833,
       0.19658178, 0.19839972, 0.19927851, 0.19996286, 0.20002141,
       0.2001568 , 0.20054144, 0.20035371, 0.20027738])
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04

### Save to Disk

In [44]:
ECCO_global_mean_sea_level.name = 'gmsl_ecco'
fname = output_dir / ('GMSL_ECCO_dailyfiles_19922017.nc')
ECCO_global_mean_sea_level.to_netcdf(fname)